In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer

from tensorflow.keras.layers import Normalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import losses 


In [ ]:
data = pd.read_csv('D:\THANH\RESEARCH\LE01\Malware_subset.csv')
y = data['Label']
features = ['Label',' Protocol',' Bwd Packet Length Min',' Down/Up Ratio',' Min Packet Length']
X = data.drop(columns=features)

In [ ]:
trans=QuantileTransformer()
test=trans.fit_transform(X)
transformer = MinMaxScaler()
X=transformer.fit_transform(test)
encoder=LabelEncoder()
y=encoder.fit_transform(data['Label'])
print(y)
print(y.shape)


[ 2  2  2 ... 10 10 10]
(834695,)


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y) 

In [ ]:
print(x_train.shape)
print(y_train.shape)

(667756, 76)
(667756,)


In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train.shape[1], 1)),    #Tạo feature maps
    tf.keras.layers.MaxPooling1D(pool_size=2),                                                                  #Giảm kích thước của feature maps
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),                                                                                  #Chuyển feature maps thành vector 1 chiều
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(11, activation='sigmoid')
  ])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                                patience=15,  # Stop training after 5 epochs with no improvement
                                verbose=1)  # Print messages during training

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Fit the model with early stopping callback
history=model.fit(x_train, y_train, batch_size=42, epochs=100,  # Adjust epochs if needed
          validation_data=(x_test, y_test),verbose=1)

c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 172s 10ms/step - accuracy: 0.5210 - loss: 1.7160 - val_accuracy: 0.5509 - val_loss: 1.6079
Epoch 2/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 235s 15ms/step - accuracy: 0.5517 - loss: 1.5944 - val_accuracy: 0.5540 - val_loss: 1.5764
Epoch 3/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 153s 10ms/step - accuracy: 0.5564 - loss: 1.5650 - val_accuracy: 0.5565 - val_loss: 1.5588
Epoch 4/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 146s 9ms/step - accuracy: 0.5585 - loss: 1.5496 - val_accuracy: 0.5606 - val_loss: 1.5438
Epoch 5/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 136s 9ms/step - accuracy: 0.5610 - loss: 1.5372 - val_accuracy: 0.5610 - val_loss: 1.5380
Epoch 6/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 136s 9ms/step - accuracy: 0.5603 - loss: 1.5331 - val_accuracy: 0.5618 - val_loss: 1.5357
Epoch 7/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 142s 9ms/step - accuracy: 0.5618 - loss: 1.5263 - val_accuracy: 0.5618 - val_loss: 1.5313
Epoch 8/100
15899/15899 ━━━━━━━━━━━━━━━━━━━━ 140s 9m

In [ ]:
loss_train = history.history['train_loss']
loss_val = history.history['val_loss']
epochs = range(1,100)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
predtrain=model.predict(x_train)
predtest=model.predict(x_test)


5217/5217 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step


array([[6.0594801e-02, 1.6050442e-01, 1.7226365e-01, ..., 1.5503424e-01,
        8.5627720e-02, 4.9966592e-01],
       [5.8814207e-09, 2.6172836e-05, 4.0901603e-05, ..., 2.5212095e-05,
        9.8788213e-09, 4.0120690e-04],
       [1.8614641e-04, 3.4489674e-03, 4.0559499e-03, ..., 3.0855425e-03,
        3.0303380e-04, 6.2777000e-03],
       ...,
       [2.3080801e-27, 1.9118083e-28, 3.0800587e-26, ..., 1.5969386e-26,
        1.9647465e-27, 1.3950024e-25],
       [1.4591918e-10, 1.3136513e-06, 1.6735752e-06, ..., 1.2316287e-06,
        1.9431254e-10, 2.9617347e-06],
       [7.4499734e-03, 8.4036263e-03, 6.4409235e-03, ..., 7.9128342e-03,
        1.0681178e-02, 1.4343543e-01]], dtype=float32)